# LayerNorm CUDA Example

This notebook demonstrates the complete process of adding a CUDA algorithm (LayerNorm) to the workspace.

## What is LayerNorm?

LayerNormalization normalizes inputs across the feature dimension:

$$\text{LayerNorm}(x) = \gamma \frac{x - \mu}{\sqrt{\sigma^2 + \epsilon}} + \beta$$

Where:
- $\mu$ is the mean across features
- $\sigma^2$ is the variance across features  
- $\gamma$ and $\beta$ are learnable parameters
- $\epsilon$ is a small constant for numerical stability

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import time

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name()}")

## Step 1: CPU Reference Implementation

Always start with a CPU implementation to understand the algorithm:

In [ ]:
def manual_layernorm(x, weight=None, bias=None, eps=1e-5):
    """
    Manual implementation to understand the algorithm
    """
    print(f"Input shape: {x.shape}")
    
    # Step 1: Calculate mean across the last dimension (features)
    mean = x.mean(dim=-1, keepdim=True)
    print(f"Mean shape: {mean.shape}")
    
    # Step 2: Calculate variance
    variance = x.var(dim=-1, keepdim=True, unbiased=False)
    print(f"Variance shape: {variance.shape}")
    
    # Step 3: Normalize
    x_norm = (x - mean) / torch.sqrt(variance + eps)
    print(f"Normalized shape: {x_norm.shape}")
    
    # Step 4: Scale and shift (if provided)
    if weight is not None:
        x_norm = x_norm * weight
        print(f"After scaling: {x_norm.shape}")
    
    if bias is not None:
        x_norm = x_norm + bias
        print(f"After bias: {x_norm.shape}")
    
    return x_norm

# Test the manual implementation
batch_size, seq_len, hidden_size = 2, 4, 6
x = torch.randn(batch_size, seq_len, hidden_size)
weight = torch.randn(hidden_size)
bias = torch.randn(hidden_size)

print("=== Manual LayerNorm ===")
output = manual_layernorm(x, weight, bias)
print(f"\nOutput shape: {output.shape}")
print(f"Output mean per feature: {output.mean(dim=(0,1))}")
print(f"Output std per feature: {output.std(dim=(0,1))}")

## Step 2: Verify Against PyTorch Implementation

In [ ]:
# Compare with PyTorch's LayerNorm
layer_norm = torch.nn.LayerNorm(hidden_size)
layer_norm.weight.data = weight.clone()
layer_norm.bias.data = bias.clone()

pytorch_output = layer_norm(x)
manual_output = manual_layernorm(x, weight, bias)

print(f"Manual implementation matches PyTorch: {torch.allclose(manual_output, pytorch_output, atol=1e-6)}")
print(f"Max difference: {torch.max(torch.abs(manual_output - pytorch_output))}")

## Step 3: Understanding CUDA Parallelization Strategy

For LayerNorm, we need to think about how to parallelize:

1. **Each sequence position** can be processed independently
2. **Within each position**, we need to:
   - Calculate mean and variance (reduction across features)
   - Apply normalization element-wise

### CUDA Kernel Design:
- **Grid**: `(batch_size, seq_len)` - one block per sequence position
- **Block**: 256 threads to process features in parallel
- **Shared Memory**: For efficient reductions

In [ ]:
# Analyze the computation pattern
def analyze_layernorm_computation(batch_size, seq_len, hidden_size):
    print(f"LayerNorm Analysis for shape [{batch_size}, {seq_len}, {hidden_size}]:")
    print(f"  Total sequence positions: {batch_size * seq_len}")
    print(f"  Features per position: {hidden_size}")
    print(f"  Total elements: {batch_size * seq_len * hidden_size:,}")
    
    # Memory analysis
    input_memory = batch_size * seq_len * hidden_size * 4  # 4 bytes per float
    stats_memory = batch_size * seq_len * 2 * 4  # mean and std
    weight_bias_memory = hidden_size * 2 * 4
    
    print(f"\nMemory Usage:")
    print(f"  Input tensor: {input_memory / 1024**2:.2f} MB")
    print(f"  Statistics (mean/std): {stats_memory / 1024**2:.2f} MB")
    print(f"  Weight/Bias: {weight_bias_memory / 1024:.2f} KB")
    
    # Parallelization strategy
    print(f"\nCUDA Parallelization:")
    print(f"  Grid size: ({batch_size}, {seq_len}) = {batch_size * seq_len} blocks")
    print(f"  Block size: 256 threads")
    print(f"  Threads per feature: {256 / hidden_size:.2f}" if hidden_size <= 256 else f"  Features per thread: {hidden_size / 256:.2f}")

# Analyze different sizes
sizes = [(2, 128, 768), (32, 512, 1024), (64, 1024, 2048)]
for size in sizes:
    analyze_layernorm_computation(*size)
    print("\n" + "="*50 + "\n")

## Step 4: Test Our CUDA Implementation (if available)

In [ ]:
# Try to import our CUDA implementation
try:
    from kernel_fusion.kernels.cuda_layernorm import layernorm, CUDALayerNorm
    from kernel_fusion.kernels.cpu_layernorm import cpu_layernorm
    CUDA_IMPL_AVAILABLE = True
    print("✅ CUDA LayerNorm implementation available!")
except ImportError as e:
    CUDA_IMPL_AVAILABLE = False
    print(f"❌ CUDA LayerNorm not available: {e}")
    print("   This is expected on CPU-only systems or before compilation")

if CUDA_IMPL_AVAILABLE:
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Testing on device: {device}")
    
    # Test our implementation
    x_test = torch.randn(4, 8, 128, device=device)
    weight_test = torch.randn(128, device=device)
    bias_test = torch.randn(128, device=device)
    
    # Our implementation
    our_output = layernorm(x_test, weight_test, bias_test)
    
    # PyTorch reference
    layer_norm_ref = torch.nn.LayerNorm(128, device=device)
    layer_norm_ref.weight.data = weight_test.clone()
    layer_norm_ref.bias.data = bias_test.clone()
    pytorch_output = layer_norm_ref(x_test)
    
    # Compare
    print(f"\nCorrectness check:")
    print(f"  Our output shape: {our_output.shape}")
    print(f"  PyTorch output shape: {pytorch_output.shape}")
    print(f"  Max difference: {torch.max(torch.abs(our_output - pytorch_output))}")
    print(f"  Outputs match: {torch.allclose(our_output, pytorch_output, atol=1e-4)}")
else:
    print("\n💡 To test the CUDA implementation:")
    print("   1. Build the Docker container with GPU support")
    print("   2. Run this notebook in the GPU environment")
    print("   3. The CUDA kernels will compile on first use")

## Step 5: Performance Analysis

In [ ]:
def benchmark_layernorm_implementations():
    """Benchmark different LayerNorm implementations"""
    
    if not torch.cuda.is_available():
        print("CUDA not available - running CPU benchmark only")
        device = torch.device('cpu')
    else:
        device = torch.device('cuda')
    
    # Test configuration
    batch_size, seq_len, hidden_size = 32, 512, 768
    x = torch.randn(batch_size, seq_len, hidden_size, device=device)
    weight = torch.randn(hidden_size, device=device)
    bias = torch.randn(hidden_size, device=device)
    
    print(f"Benchmarking LayerNorm on {device}")
    print(f"Input shape: [{batch_size}, {seq_len}, {hidden_size}]")
    print("\nImplementations to test:")
    
    implementations = {}
    
    # PyTorch implementation
    layer_norm_pytorch = torch.nn.LayerNorm(hidden_size, device=device)
    layer_norm_pytorch.weight.data = weight.clone()
    layer_norm_pytorch.bias.data = bias.clone()
    implementations['PyTorch'] = lambda: layer_norm_pytorch(x)
    
    # Manual implementation  
    implementations['Manual'] = lambda: manual_layernorm(x, weight, bias)
    
    # Our CUDA implementation (if available)
    if CUDA_IMPL_AVAILABLE:
        implementations['Our CUDA'] = lambda: layernorm(x, weight, bias)
    
    # Benchmark each implementation
    results = {}
    for name, func in implementations.items():
        print(f"  - {name}")
        
        # Warmup
        for _ in range(5):
            _ = func()
        
        if device.type == 'cuda':
            torch.cuda.synchronize()
        
        # Benchmark
        times = []
        for _ in range(20):
            start = time.time()
            _ = func()
            if device.type == 'cuda':
                torch.cuda.synchronize()
            times.append(time.time() - start)
        
        avg_time = np.mean(times[5:]) * 1000  # Skip first 5, convert to ms
        results[name] = avg_time
    
    # Display results
    print("\n" + "="*40)
    print("Benchmark Results:")
    print("="*40)
    
    baseline = results['PyTorch']
    for name, time_ms in results.items():
        speedup = baseline / time_ms
        print(f"{name:<15}: {time_ms:6.2f}ms  ({speedup:4.2f}x)")
    
    return results

# Run benchmark
benchmark_results = benchmark_layernorm_implementations()

## Step 6: Visualize LayerNorm Effects

In [ ]:
# Visualize what LayerNorm does to the data
def visualize_layernorm_effects():
    # Create sample data with different scales
    torch.manual_seed(42)
    batch_size, seq_len, hidden_size = 1, 1, 1000
    
    # Create data with different means and scales
    x1 = torch.randn(batch_size, seq_len, hidden_size) * 0.1 + 2.0   # Small variance, positive mean
    x2 = torch.randn(batch_size, seq_len, hidden_size) * 5.0 - 1.0   # Large variance, negative mean
    x3 = torch.randn(batch_size, seq_len, hidden_size) * 1.0 + 0.0   # Normal distribution
    
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    
    # Plot original distributions
    axes[0, 0].hist(x1.flatten().numpy(), bins=50, alpha=0.7, color='red')
    axes[0, 0].set_title('Original: Small var, +mean')
    axes[0, 0].set_ylabel('Frequency')
    
    axes[0, 1].hist(x2.flatten().numpy(), bins=50, alpha=0.7, color='blue')
    axes[0, 1].set_title('Original: Large var, -mean')
    
    axes[0, 2].hist(x3.flatten().numpy(), bins=50, alpha=0.7, color='green')
    axes[0, 2].set_title('Original: Normal distribution')
    
    # Apply LayerNorm
    y1 = manual_layernorm(x1)
    y2 = manual_layernorm(x2)
    y3 = manual_layernorm(x3)
    
    # Plot normalized distributions
    axes[1, 0].hist(y1.flatten().numpy(), bins=50, alpha=0.7, color='red')
    axes[1, 0].set_title('After LayerNorm')
    axes[1, 0].set_xlabel('Value')
    axes[1, 0].set_ylabel('Frequency')
    
    axes[1, 1].hist(y2.flatten().numpy(), bins=50, alpha=0.7, color='blue')
    axes[1, 1].set_title('After LayerNorm')
    axes[1, 1].set_xlabel('Value')
    
    axes[1, 2].hist(y3.flatten().numpy(), bins=50, alpha=0.7, color='green')
    axes[1, 2].set_title('After LayerNorm')
    axes[1, 2].set_xlabel('Value')
    
    plt.tight_layout()
    plt.show()
    
    # Print statistics
    print("Statistics before LayerNorm:")
    print(f"  Dataset 1: mean={x1.mean():.3f}, std={x1.std():.3f}")
    print(f"  Dataset 2: mean={x2.mean():.3f}, std={x2.std():.3f}")
    print(f"  Dataset 3: mean={x3.mean():.3f}, std={x3.std():.3f}")
    
    print("\nStatistics after LayerNorm:")
    print(f"  Dataset 1: mean={y1.mean():.3f}, std={y1.std():.3f}")
    print(f"  Dataset 2: mean={y2.mean():.3f}, std={y2.std():.3f}")
    print(f"  Dataset 3: mean={y3.mean():.3f}, std={y3.std():.3f}")

visualize_layernorm_effects()

## Summary: LayerNorm CUDA Development Process

We've demonstrated the complete process of adding a CUDA algorithm:

### ✅ **What we accomplished:**

1. **📊 Algorithm Understanding**: Broke down LayerNorm mathematically
2. **💻 CPU Prototype**: Implemented and verified correctness
3. **🚀 CUDA Design**: Planned parallelization strategy
4. **⚡ Performance Analysis**: Benchmarked implementations
5. **🧪 Testing**: Created comprehensive test suite
6. **📈 Visualization**: Understood the algorithm's effects

### 🔧 **Files Created:**
- `kernel_fusion/kernels/cpu_layernorm.py` - CPU reference
- `kernel_fusion/kernels/cuda_layernorm.py` - CUDA implementation
- `tests/test_layernorm.py` - Test suite
- This notebook - Documentation and examples

### 🎯 **Key CUDA Concepts Demonstrated:**
- **Grid/Block Organization**: One block per sequence position
- **Shared Memory**: For efficient reductions
- **Memory Coalescing**: Accessing consecutive elements
- **Fallback Mechanisms**: CPU implementation when CUDA fails

### 🚀 **Next Steps:**
1. **Compile and test** in GPU environment
2. **Profile and optimize** using nsys/ncu
3. **Add more kernels** following this pattern
4. **Benchmark against** production libraries

This process can be applied to any CUDA algorithm you want to add to the workspace!